# **Teste 01**

- **Condição de teste**: "*historico.csv*" vazio.
- **Ação no teste**: Executá-do.

## Resultados observado = Resultado desejado
- O arquivo é escrito em .csv com cabeçalho e em ordem de acordo com a consulta SQL.

### historico.csv

In [ ]:
BEHAVIOR,STATUS,PARAMETER_IDENTIFIER,ACTION_ID,TEST_ID,USER_ID,LAST_UPDATE_TIME,OP,MMC,OPERATION_ID,XML_NAME
CONFIG,PASSED,A,32551956,37179,9994,2025-02-25 18:54:34.983000,50839087,1597,3140952,1.0
CONFIG,PASSED,B,32551959,37179,9994,2025-02-25 18:54:34.983000,50839087,1597,3140952,1.0
CONFIG,PASSED,C,32551961,37179,9994,2025-02-25 18:54:34.983000,50839087,1597,3140952,1.0
VALIDATE,FAILED,D,32551965,37179,9994,2025-02-25 18:54:34.983000,50839087,1597,3140952,1.0
VALIDATE,FAILED,E,32551955,37179,9994,2025-02-25 18:54:34.983000,50839087,1597,3140952,1.0
VALIDATE,FAILED,F,32551958,37179,9994,2025-02-25 18:54:34.983000,50839087,1597,3140952,1.0
VALIDATE,PASSED,G,32551964,37179,9994,2025-02-25 18:54:34.983000,50839087,1597,3140952,1.0
VALIDATE,,H,32551963,37179,9994,2025-02-25 18:54:34.983000,50839087,1597,3140952,1.0
VALIDATE,,I,32551957,37179,9994,2025-02-25 18:54:34.983000,50839087,1597,3140952,1.0
VALIDATE,,J,32551960,37179,9994,2025-02-25 18:54:34.983000,50839087,1597,3140952,1.0
VALIDATE,,H,32551963,37179,9994,2025-02-25 18:54:34.983000,50839087,1597,3140952,1.0
VALIDATE,,I,32551957,37179,9994,2025-02-25 18:54:34.983000,50839087,1597,3140952,1.0
VALIDATE,,J,32551960,37179,9994,2025-02-25 18:54:34.983000,50839087,1597,3140952,1.0


### Código usado

In [ ]:
# O código está baseado no meu algoritmo, no Obsidian. Os comentários acima dos blocos descrevem uma etapa do fluxo e os comentários após códigos são para auxiliar no entendimento.

# Início
import cx_Oracle
import csv
import os

# Retângulo 01
consulta = """
SELECT * FROM (
    SELECT its_actions.behavior,
           its_actions.status,
           its_actions.parameter_identifier,
           its_actions.action_id,
           AB.test_id,
           AB.user_id,
           AB.last_update_time,
           AB.op,
           AB.mmc,
           AB.operation_id,
           AB.xml_name
    FROM its_actions
    FULL OUTER JOIN (
        SELECT its_test.*,
               its_operations.operation_id,
               its_operations.xml_name
        FROM its_test
        FULL OUTER JOIN its_operations ON its_test.test_id = its_operations.test_id
        WHERE its_test.last_update_time IS NOT NULL
    ) AB ON its_actions.operation_id = AB.operation_id
    WHERE TEST_ID is not null
    ORDER BY LAST_UPDATE_TIME DESC, AB.xml_name ASC, its_actions.parameter_identifier ASC
) WHERE ROWNUM <= 10
""" # nao pode possuir ponto e vírgula no final, framework não compreende
cursor_Oracle = cx_Oracle.connect(user='its_man', password='maint#its06', dsn=cx_Oracle.makedsn('ora-qa37', 1521, service_name='qa37')).cursor() # .cursor cria um objeto que permite executar comandos SQL e gerenciar os resultados de consultas

# Retângulo 02
cursor_Oracle.execute(consulta) # consulta é feita
armazenagem_intermediaria = cursor_Oracle.fetchall() # arnazenagem intermediaria dos resultados

# Condicional 1
if (os.path.getsize("./historico.csv") == 0):

# Condicional 1 "Sim"
    with open ('historico.csv', 'w', newline='') as documento: # documento criado/sobrescrito
        csv.writer(documento).writerow(coluna[0] for coluna in cursor_Oracle.description) # escreve cabeçalho
        csv.writer(documento).writerows(armazenagem_intermediaria) # escreve as linhas com base na armazenagem intermediaria. Linah 14 garante quebra de linha entre infos.
        print("\n\n ele é zero, sobrescrito \n\n") # controle. Apagar
else:

# Condicional 1 "Não"
    with open ('historico.csv', 'r+', newline='', encoding='utf-8') as documento:
        i = 0
        lista_documento = list(csv.reader(documento))

        for linha in armazenagem_intermediaria:
            i += 1
            print(f'Comparando "{linha[1]}" (linha {i+1} do servidor) com "{lista_documento[i][1]}" (do histórico)...') 

            if (int(linha[3]) == int(lista_documento[i][3])): # Condicional 2 | Compara as chaves primarias ACTION_ID        
                if (linha[1] == lista_documento[i][1]): # Condicional 2 "Sim" | # Condicional 3 | # compara STATUS
                    print("São iguais. Ignorar.\n") # Condicional 3 "Sim"
                else:
                    print("Não são iguais! Salvar\n") # Condicional 3 "Não"
                    csv.writer(documento).writerow(linha) 
            else:
                print(f'Chave primária ACTION_ID da linha bd ({linha[3]}) diferentes para esta linha em hist ({lista_documento[i][3]}). Ignorar\n') # Condicional 2 "Não"    
        
# Não presente. Apenas para fechar.
cursor_Oracle.close()
cx_Oracle.connect(user='its_man', password='maint#its06', dsn=cx_Oracle.makedsn('ora-qa37', 1521, service_name='qa37')).close()

### Saída Terminal

In [ ]:
Comparando "PASSED" (linha 2 do servidor) com "PASSED" (do histórico)...
São iguais. Ignorar.

Comparando "PASSED" (linha 3 do servidor) com "PASSED" (do histórico)...
São iguais. Ignorar.

Comparando "PASSED" (linha 4 do servidor) com "PASSED" (do histórico)...
São iguais. Ignorar.

Comparando "FAILED" (linha 5 do servidor) com "FAILED" (do histórico)...
São iguais. Ignorar.

Comparando "FAILED" (linha 6 do servidor) com "FAILED" (do histórico)...
São iguais. Ignorar.

Comparando "FAILED" (linha 7 do servidor) com "FAILED" (do histórico)...
São iguais. Ignorar.

Comparando "PASSED" (linha 8 do servidor) com "PASSED" (do histórico)...
São iguais. Ignorar.

Comparando "None" (linha 9 do servidor) com "" (do histórico)...
Não são iguais! Salvar

Comparando "None" (linha 10 do servidor) com "" (do histórico)...
Não são iguais! Salvar

Comparando "None" (linha 11 do servidor) com "" (do histórico)...
Não são iguais! Salvar

# __________________________________________________________________________________________

Problemas em potencial: 
- Está dependendo de ordem?
<br>**R:** <span style="color:#C82F4B">Sim, preciso mudar.</span>
<br>**R:** <span style="color:#6990C3">Preciso fazer depender de ID.</span>

- E se eu alterar o tamanho da análise?

- E se a lista do historico começar com elementos diferentes do BD?